In [0]:
%sh
pip install --upgrade pip
pip install kafka-python

In [0]:
import pyspark.pandas as ps
import requests
from datetime import datetime, date, timedelta

global termos
termos = [
        "sequenciamento DNA",
        "genomica",
        "genoma",
        "terapia genetica"
    ]

def carrega_noticias():
    """
    Busca as últimas notícias com base nos termos pré-definidos e as retorna em formato json
    """
    ultima_data = (date.today() - timedelta(days=30)).strftime('%Y-%m-%d')
    dados = list()
    
    chave_api = dbutils.fs.head('/FileStore/tables/projeto/token.txt')

    for termo in termos:
        termo_busca = termo.replace(" ", "+")
        url = f'https://newsapi.org/v2/everything?q={termo_busca}&language=pt&from={ultima_data}&sortBy=publishedAt&apiKey={chave_api}'
        # &country=br
        r = requests.get(url)
        resultados = r.json()['articles']
        for resultado in resultados:
            resultado['searchTerm'] = termo
            resultado['sourceId'] = resultado['source']['id']
            resultado['source'] = resultado['source']['name']
            if resultado['publishedAt'].startswith('2023'):
                dados.append(resultado)

    return dados


In [0]:
import json
from kafka import KafkaProducer

produtor = KafkaProducer(bootstrap_servers=["localhost:9092"])      # declara o produtor
topico = "1st_topic_news"                                           # nome do tópico

In [0]:
# Cria o arquivo news.parquet
lista_noticias = carrega_noticias()
df_news = ps.DataFrame(lista_noticias)
df_news.to_parquet('/FileStore/tables/projeto/news.parquet')

In [0]:
import time

while True:
    lista_noticias = carrega_noticias()
    lista_noticias = lista_noticias[:len(lista_noticias)//5]
    print(f"Total de notícias a processar: {len(lista_noticias)}")

    for i, noticia in enumerate(lista_noticias):
        mensagem = json.dumps(noticia).encode('utf-8')
        produtor.send(topico, mensagem)
        print(f"Notícia {i+1} de {len(lista_noticias)} enviada!")
        time.sleep(5)
    
    time.sleep(3600)

Total de notícias a processar: 4
Notícia 1 de 4 enviada!
Notícia 2 de 4 enviada!
Notícia 3 de 4 enviada!
Notícia 4 de 4 enviada!
